In [1]:
import numpy as np
import tensorflow as tf

# set parameters ###############################################################
initial_burning_epoch = 1000
epoch = 100
lr_rate = 1e-2
copy_period = 5
epsilon_0 = 1e-5
# set parameters ###############################################################

# state
states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
N_STATES = len(states)

# action
actions = [0, 1, 2, 3] 
N_ACTIONS = len(actions)

# transition probabilities
P = np.empty((N_STATES, N_ACTIONS, N_STATES))

#                0   1   2   3   4   5   6   7   8   9  10
P[ 0, 0, :] = [ .9,  0,  0,  0, .1,  0,  0,  0,  0,  0,  0]
P[ 0, 1, :] = [ .1, .8,  0,  0, .1,  0,  0,  0,  0,  0,  0]
P[ 0, 2, :] = [ .9, .1,  0,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 0, 3, :] = [ .1, .1,  0,  0, .8,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 1, 0, :] = [ .8, .2,  0,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 1, :] = [  0, .2, .8,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 2, :] = [ .1, .8, .1,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 3, :] = [ .1, .8, .1,  0,  0,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 2, 0, :] = [  0, .8, .1,  0,  0, .1,  0,  0,  0,  0,  0]
P[ 2, 1, :] = [  0,  0, .1, .8,  0, .1,  0,  0,  0,  0,  0]
P[ 2, 2, :] = [  0, .1, .8, .1,  0,  0,  0,  0,  0,  0,  0]
P[ 2, 3, :] = [  0, .1,  0, .1,  0, .8,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 3, 0, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 1, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 2, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 3, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 4, 0, :] = [ .1,  0,  0,  0, .8,  0,  0, .1,  0,  0,  0]
P[ 4, 1, :] = [ .1,  0,  0,  0, .8,  0,  0, .1,  0,  0,  0]
P[ 4, 2, :] = [ .8,  0,  0,  0, .2,  0,  0,  0,  0,  0,  0]
P[ 4, 3, :] = [  0,  0,  0,  0, .2,  0,  0, .8,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 5, 0, :] = [  0,  0, .1,  0,  0, .8,  0,  0,  0, .1,  0]
P[ 5, 1, :] = [  0,  0, .1,  0,  0,  0, .8,  0,  0, .1,  0]
P[ 5, 2, :] = [  0,  0, .8,  0,  0, .1, .1,  0,  0,  0,  0]
P[ 5, 3, :] = [  0,  0,  0,  0,  0, .1, .1,  0,  0, .8,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 6, 0, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 1, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 2, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 3, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 7, 0, :] = [  0,  0,  0,  0, .1,  0,  0, .9,  0,  0,  0]
P[ 7, 1, :] = [  0,  0,  0,  0, .1,  0,  0, .1, .8,  0,  0]
P[ 7, 2, :] = [  0,  0,  0,  0, .8,  0,  0, .1, .1,  0,  0]
P[ 7, 3, :] = [  0,  0,  0,  0,  0,  0,  0, .9, .1,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 8, 0, :] = [  0,  0,  0,  0,  0,  0,  0, .8, .2,  0,  0]
P[ 8, 1, :] = [  0,  0,  0,  0,  0,  0,  0,  0, .2, .8,  0]
P[ 8, 2, :] = [  0,  0,  0,  0,  0,  0,  0, .1, .8, .1,  0]
P[ 8, 3, :] = [  0,  0,  0,  0,  0,  0,  0, .1, .8, .1,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 9, 0, :] = [  0,  0,  0,  0,  0, .1,  0,  0, .8, .1,  0]
P[ 9, 1, :] = [  0,  0,  0,  0,  0, .1,  0,  0,  0, .1, .8]
P[ 9, 2, :] = [  0,  0,  0,  0,  0, .8,  0,  0, .1,  0, .1]
P[ 9, 3, :] = [  0,  0,  0,  0,  0,  0,  0,  0, .1, .8, .1]

#                0   1   2   3   4   5   6   7   8   9  10
P[10, 0, :] = [  0,  0,  0,  0,  0,  0, .1,  0,  0, .8, .1]
P[10, 1, :] = [  0,  0,  0,  0,  0,  0, .1,  0,  0,  0, .9]
P[10, 2, :] = [  0,  0,  0,  0,  0,  0, .8,  0,  0, .1, .1]
P[10, 3, :] = [  0,  0,  0,  0,  0,  0,  0,  0,  0, .1, .9]

# rewards
R = -0.02 * np.ones((N_STATES, N_ACTIONS)) 
R[3,:] = 1.
R[6,:] = -1.
    
# discount factor
gamma = 0.99

# policy
if 0: 
    # bad policy 
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,0,1]
    policy[5,:] = [0,1,0,0]
    policy[6,:] = [0,1,0,0]
    policy[7,:] = [0,1,0,0]
    policy[8,:] = [0,1,0,0]
    policy[9,:] = [0,0,1,0]
    policy[10,:] = [0,0,1,0]
elif 0: 
    # random policy
    policy = 0.25*np.ones((N_STATES, N_ACTIONS))
elif 0: 
    # optimal policy 
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,1,0]
    policy[5,:] = [0,0,1,0]
    policy[6,:] = [0,0,1,0]
    policy[7,:] = [0,0,1,0]
    policy[8,:] = [1,0,0,0]
    policy[9,:] = [1,0,0,0]
    policy[10,:] = [1,0,0,0]
elif 1: 
    # optimal policy + noise 
    # we use optimal policy with probability 1/(1+ep)
    # we use random policy with probability ep/(1+ep)
    ep = 0.1
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,1,0]
    policy[5,:] = [0,0,1,0]
    policy[6,:] = [0,0,1,0]
    policy[7,:] = [0,0,1,0]
    policy[8,:] = [1,0,0,0]
    policy[9,:] = [1,0,0,0]
    policy[10,:] = [1,0,0,0]
    policy = policy + (ep/4)*np.ones((N_STATES, N_ACTIONS))
    policy = policy / np.sum(policy, axis=1).reshape((N_STATES,1))


In [2]:
" TD "
# https://www.tensorflow.org/api_docs/python/tf/placeholder
state = tf.placeholder(tf.int32, shape=[None], name='state')
action = tf.placeholder(tf.int32, shape=[None], name='action')
reward = tf.placeholder(tf.float32, shape=[None], name='reward')
next_state = tf.placeholder(tf.int32, shape=[None], name='next_state')
next_action = tf.placeholder(tf.int32, shape=[None], name='next_action')
done_holder = tf.placeholder(tf.bool, shape=[None], name='done_holder')

#어떤 구조를 만들어주기 위해 쓰는 것. 그 구조를 활용할때는 feed_dict로 각각의 부분에 해당하는 값을 대응시키는 것이 필요
#https://gdyoon.tistory.com/5

# https://www.tensorflow.org/api_docs/python/tf/one_hot
# one_hot(x,y) x의 내용 개수 만큼 row를 만들어주고, y값 만큼 column을 만들어주고, 그리고 x의 내용 번째 값을 1, 아닌 것을 0으로
# one_hot([2,0,1],3) = [[0,0,1],[1,0,0],[0,1,0]]
state_one_hot = tf.one_hot(state, N_STATES)
action_one_hot = tf.one_hot(action, N_ACTIONS)
next_state_one_hot = tf.one_hot(next_state, N_STATES)
next_action_one_hot = tf.one_hot(next_action, N_ACTIONS)

# https://www.tensorflow.org/api_docs/python/tf/concat
#데이터 합치기
#t1= [[1,0,0],[0,1,0],[0,0,1]]
#t2=[[2,0,0],[0,2,0],[0,0,2]]
#concat([t1,t2],0)=[[1,0,0],[0,1,0],[0,0,1],[2,0,0],[0,2,0],[0,0,2]]
#concat([t1,t2],1)=[[1,0,0,2,0,0],[0,1,0,0,2,0],[0,0,1,0,0,2]]
state_action_one_hot = tf.concat([state_one_hot, action_one_hot], 1)
next_state_action_one_hot = tf.concat([next_state_one_hot, next_action_one_hot], 1)


with tf.Session() as sess:
    print(sess.run(state_action_one_hot, feed_dict={state: [1],action: [2]}))
    print(sess.run(state_one_hot, feed_dict={state:[3]}))
    print(sess.run(action_one_hot, feed_dict={action:[2]}))
    
with tf.Session() as sess:
    print(sess.run(next_state_action_one_hot, feed_dict={next_state: [0],next_action: [2]}))

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 1. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [3]:
# Previously
# Q = np.zeros((N_STATES, N_ACTIONS))
# Q[3, :] = 1.
# Q[6, :] = -1.

# state s: one hot encode
# action a: one hot encode
# Q[s,a]: [s as one hot encode, a as one hot encode] @ W 

with tf.variable_scope('main_net', reuse = tf.AUTO_REUSE) as scope:
    W = tf.get_variable(name='W', \
                        shape=[N_STATES + N_ACTIONS, 1], \
                        dtype=tf.float32, \
                        initializer=tf.random_uniform_initializer(-1.0, 1.0))
    AA = tf.get_variable(name='AA', shape=[N_ACTIONS,1],dtype=tf.float32)
    
with tf.variable_scope('target_net') as scope:
    W_target = tf.get_variable(name='W_target', \
                        shape=[N_STATES + N_ACTIONS, 1], \
                        dtype=tf.float32, \
                        initializer=tf.random_uniform_initializer(-1.0, 1.0))

#커서 위치에 있는 것 주석으로 만들기 : ctrl+/ - 여러줄 주석도 한꺼버넹 할 수 있음
    
# init_op = tf.global_variables_initializer()

# with tf.Session() as sess:
#     sess.run(init_op)
#     print(sess.run(W))
#     print(sess.run(W_target))
    
#변수를 사용하려면 변수 intializer를 해줘야한다. 안그러면 안됨. 그리고 저 init_op 보다 아래에서 만들어진 변수는 초기화가 안되는 듯.
#참고 : https://3months.tistory.com/68

#tf.get_collection - 현재 만들어져있는 variable을 리스트로 만들라는 것
# variable copy
# from_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'main_net')
# # print(from_vars)
# # [<tf.Variable 'main_net/W:0' shape=(15, 1) dtype=float32_ref>]

# to_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'target_net')
# # print(to_vars)
# # [<tf.Variable 'target_net/W_target:0' shape=(15, 1) dtype=float32_ref>]

# copy_ops = [to_vars[i].assign(from_vars[i]) for i in range(len(from_vars))]

# i=[1]
# neww_action_one_hot = tf.one_hot(i, N_ACTIONS)
#neww_state_action_one_hot = tf.concat([next_state_one_hot, neww_action_one_hot], 1)


for j in range(N_ACTIONS) :
    neww_action_one_hot = tf.one_hot([j], N_ACTIONS)
    neww_state_action_one_hot = tf.concat([next_state_one_hot, neww_action_one_hot], 1)
    AA[j]= tf.matmul(neww_state_action_one_hot, W)


with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(W))
    print(sess.run(neww_action_one_hot))
#    print(sess.run(neww_state_action_one_hot, feed_dict = {next_state : [1]}))
    print(sess.run(AA, feed_dict = {next_state : [1]}))
    


TypeError: 'RefVariable' object does not support item assignment